# Projeto Parcial Ciência de Dados (N1)

| Nome | RA |
| ---- | -- |
| Enzo Ferroni | 10417100 |
| Luiz Gabriel Profirio Mendes | 10382703 |



## Escolha dos Modelos

---

* Regressão Linear  
JUSTIFICATIVA

* KNN  
JUSTIFICATIVA

## Pré-processamento básico

---

In [14]:
# Importa bibliotecas
import pandas as pd

In [15]:
# Lê CSV
df = pd.read_csv("spotify-2023.csv", encoding="latin1")

# Foi necessário usar o 'encoding' no bloco acima pois estava com o erro 
# UnicodeDecodeError: 'utf-8' codec can't decode bytes in position 7250-7251: invalid continuation byte


In [16]:
# Exibe algumas informações básicas sobre o dataset.
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 953 entries, 0 to 952
Data columns (total 24 columns):
 #   Column                Non-Null Count  Dtype 
---  ------                --------------  ----- 
 0   track_name            953 non-null    object
 1   artist(s)_name        953 non-null    object
 2   artist_count          953 non-null    int64 
 3   released_year         953 non-null    int64 
 4   released_month        953 non-null    int64 
 5   released_day          953 non-null    int64 
 6   in_spotify_playlists  953 non-null    int64 
 7   in_spotify_charts     953 non-null    int64 
 8   streams               953 non-null    object
 9   in_apple_playlists    953 non-null    int64 
 10  in_apple_charts       953 non-null    int64 
 11  in_deezer_playlists   953 non-null    object
 12  in_deezer_charts      953 non-null    int64 
 13  in_shazam_charts      903 non-null    object
 14  bpm                   953 non-null    int64 
 15  key                   858 non-null    ob

In [17]:
# Verificando os valores presentes em key
print(f"Value counts para a coluna 'key':")
value_counts = df['key'].value_counts()
display(value_counts)
print("\n")

Value counts para a coluna 'key':


key
C#    120
G      96
G#     91
F      89
B      81
D      81
A      75
F#     73
E      62
A#     57
D#     33
Name: count, dtype: int64

In [18]:
# Remove linhas onde key é null
df = df.dropna(subset=['key'])

# Exibe algumas informações básicas sobre o dataset.
df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 858 entries, 0 to 952
Data columns (total 24 columns):
 #   Column                Non-Null Count  Dtype 
---  ------                --------------  ----- 
 0   track_name            858 non-null    object
 1   artist(s)_name        858 non-null    object
 2   artist_count          858 non-null    int64 
 3   released_year         858 non-null    int64 
 4   released_month        858 non-null    int64 
 5   released_day          858 non-null    int64 
 6   in_spotify_playlists  858 non-null    int64 
 7   in_spotify_charts     858 non-null    int64 
 8   streams               858 non-null    object
 9   in_apple_playlists    858 non-null    int64 
 10  in_apple_charts       858 non-null    int64 
 11  in_deezer_playlists   858 non-null    object
 12  in_deezer_charts      858 non-null    int64 
 13  in_shazam_charts      817 non-null    object
 14  bpm                   858 non-null    int64 
 15  key                   858 non-null    object


In [19]:
# Remove colunas de serviços que não serão cobertos em nossa análise.
df = df.drop(columns=['in_apple_playlists', 'in_apple_charts', 'in_deezer_playlists', 'in_deezer_charts', 'in_shazam_charts'])

# Exibe algumas informações básicas sobre o dataset.
df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 858 entries, 0 to 952
Data columns (total 19 columns):
 #   Column                Non-Null Count  Dtype 
---  ------                --------------  ----- 
 0   track_name            858 non-null    object
 1   artist(s)_name        858 non-null    object
 2   artist_count          858 non-null    int64 
 3   released_year         858 non-null    int64 
 4   released_month        858 non-null    int64 
 5   released_day          858 non-null    int64 
 6   in_spotify_playlists  858 non-null    int64 
 7   in_spotify_charts     858 non-null    int64 
 8   streams               858 non-null    object
 9   bpm                   858 non-null    int64 
 10  key                   858 non-null    object
 11  mode                  858 non-null    object
 12  danceability_%        858 non-null    int64 
 13  valence_%             858 non-null    int64 
 14  energy_%              858 non-null    int64 
 15  acousticness_%        858 non-null    int64 


In [ ]:
# Remove linha da música Love Grows (Where My Rosemary Goes), pois há inconsistência nos dados.
index = df[df['track_name'] == "Love Grows (Where My Rosemary Goes)"].index
df = df.drop(index)

df.info()


<class 'pandas.core.frame.DataFrame'>
Index: 857 entries, 0 to 952
Data columns (total 19 columns):
 #   Column                Non-Null Count  Dtype 
---  ------                --------------  ----- 
 0   track_name            857 non-null    object
 1   artist(s)_name        857 non-null    object
 2   artist_count          857 non-null    int64 
 3   released_year         857 non-null    int64 
 4   released_month        857 non-null    int64 
 5   released_day          857 non-null    int64 
 6   in_spotify_playlists  857 non-null    int64 
 7   in_spotify_charts     857 non-null    int64 
 8   streams               857 non-null    object
 9   bpm                   857 non-null    int64 
 10  key                   857 non-null    object
 11  mode                  857 non-null    object
 12  danceability_%        857 non-null    int64 
 13  valence_%             857 non-null    int64 
 14  energy_%              857 non-null    int64 
 15  acousticness_%        857 non-null    int64 


In [ ]:
# Converte streams para numérico
df['streams'] = pd.to_numeric(df['streams'])

df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 857 entries, 0 to 952
Data columns (total 19 columns):
 #   Column                Non-Null Count  Dtype 
---  ------                --------------  ----- 
 0   track_name            857 non-null    object
 1   artist(s)_name        857 non-null    object
 2   artist_count          857 non-null    int64 
 3   released_year         857 non-null    int64 
 4   released_month        857 non-null    int64 
 5   released_day          857 non-null    int64 
 6   in_spotify_playlists  857 non-null    int64 
 7   in_spotify_charts     857 non-null    int64 
 8   streams               857 non-null    int64 
 9   bpm                   857 non-null    int64 
 10  key                   857 non-null    object
 11  mode                  857 non-null    object
 12  danceability_%        857 non-null    int64 
 13  valence_%             857 non-null    int64 
 14  energy_%              857 non-null    int64 
 15  acousticness_%        857 non-null    int64 


## Implementação

---

### Regressão Linear

## Métricas

---

## Comparação

---

## Conclusão preliminar

---